In [1]:
!python -V

Python 3.12.3


In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1717081967461, experiment_id='1', last_update_time=1717081967461, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

artifact_location='/workspaces/MLOpps24/02-experiment-tracking/mlruns/1': Provides the artifact location in the MLFlow URI


In [6]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df2 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

## Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

In [7]:
print("There are a total of", len(df1.columns),"columns.")

There are a total of 19 columns.


## Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

    32.59
    42.59
    52.59
    62.59

    

In [8]:
df1["duration"] = df1["tpep_dropoff_datetime"] - df1['tpep_pickup_datetime']
df1.duration = df1.duration.apply(lambda td: td.total_seconds() / 60)
df1['PUMonth'] = df1['tpep_pickup_datetime'].dt.month 


In [9]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,PUMonth
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333,1
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667,1
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000,1
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667,1
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333,1


In [10]:
df1 = df1[df1.PUMonth==1]

dur_std = df1['duration'].std() 

print("The Standard Deviation of Trip Duration is", round(dur_std,2) )

The Standard Deviation of Trip Duration is 42.59


## Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [11]:
df1_filter = df1[(df1.duration >= 1) & (df1.duration <= 60)]

In [12]:
print("The original dataset was",len(df1),"rows, and the newer dataset was", len(df1_filter),"rows. So we are left with",round(len(df1_filter)/len(df1),2)*100 ,"percent of the original dataset.")

The original dataset was 3066718 rows, and the newer dataset was 3009136 rows. So we are left with 98.0 percent of the original dataset.


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

    2
    155
    345
    515
    715


In [13]:
# Convert columns to 'category' if they have a limited number of unique values
df1_filter[["PULocationID","DOLocationID"]] = df1_filter[["PULocationID","DOLocationID"]].astype(str)



/tmp/ipykernel_54890/2605247708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_filter[["PULocationID","DOLocationID"]] = df1_filter[["PULocationID","DOLocationID"]].astype(str)


In [14]:

q4_one_hot = df1_filter[["PULocationID", "DOLocationID"]]
q4_one_hot = q4_one_hot.to_dict(orient = "records")
v = DictVectorizer()


In [15]:
X = v.fit_transform(q4_one_hot)

In [16]:
X.shape

(3009136, 515)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters
    Calculate the RMSE of the model on the training data

What's the RMSE on train?



In [17]:
X_train = X
y_train = df1_filter['duration'].values

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

In [18]:
mean_squared_error(y_train, y_pred, squared=False)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649229612654799

## Put the functions to read in the data all together. 

In [19]:
def taxi_data_prep(filename, month, low_bnd,upp_bnd):
     df = pd.read_parquet(filename)
     # Start prep of duration
     df["duration"] = df["tpep_dropoff_datetime"] - df['tpep_pickup_datetime']
     df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
     df['PUMonth'] = df['tpep_pickup_datetime'].dt.month 

     # Filter out the month and durations
     df = df[df.PUMonth==month]
     df_filter = df[(df.duration >= low_bnd) & (df.duration <= upp_bnd)]
     # Convert columns to 'category' if they have a limited number of unique values
     df_filter[["PULocationID","DOLocationID"]] = df_filter[["PULocationID","DOLocationID"]].astype(str)

     return df_filter

In [20]:
df_train = taxi_data_prep("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet",
                1, 1,60)
df_val = taxi_data_prep("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet",
                2, 1,60)

/tmp/ipykernel_54890/3872572736.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[["PULocationID","DOLocationID"]] = df_filter[["PULocationID","DOLocationID"]].astype(str)
/tmp/ipykernel_54890/3872572736.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filter[["PULocationID","DOLocationID"]] = df_filter[["PULocationID","DOLocationID"]].astype(str)


## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [21]:
v = DictVectorizer()

# Create one hot encoding from df
categorical = ["PULocationID", "DOLocationID"]
train_dicts = df_train[categorical].to_dict(orient = "records")
val_dicts = df_val[categorical].to_dict(orient = "records")

X_train = v.fit_transform(train_dicts)
X_val = v.transform(val_dicts)

In [22]:
# Statistical Model Prep
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.811824786042341

# Save output and model into the model folder

In [24]:
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((v,lr),f_out)

# What do we do to update this? 

We can start tracking a new history using MLFlow
- Features like `mlflow.start_run()` can be incredibly helpful in this process. Let's try to update our model by using lasso. 

In [25]:
lasso = Lasso(0.1)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)

mean_squared_error(y_val, y_pred, squared = False) 

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


8.75055890627609

This did a bit worse so let's try to update this with MLFlow's Processes!

In [26]:
with mlflow.start_run():
    # Log developer
    mlflow.set_tag("developer", "Emmanuel")
    # Set training and validation data pths 
    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.01
    # Log alpha parameters to evaluate model effectiveness
    mlflow.log_param("alpha", alpha)
    lasso = Lasso(alpha)
    lasso.fit(X_train, y_train)
    
    y_pred = lasso.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared = False)
    mlflow.log_metric("rmse", rmse)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Hyperparameter Optimization with XGBoost 



In [27]:
import xgboost as xgb

# fMin is the object that minimizes the error obkects 
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [28]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [29]:
def objective(params):
    """
    params: Set of parameters for XGBOOST that get logged to mlflow and generates booster object. 
    """
    with mlflow.start_run():
        mlflow.set_tag("model","xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            # max iterations 
            num_boost_round=200,
            # Validation set gets used to control the optimization algorithm
            evals=[(valid,"validation")], 
            # if there are 50 rounds without better optimization then it stops 
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status':STATUS_OK}


        

In [31]:
search_space = {
    # controls the depth of the trees: 4-100 levels; returns a real number but we want an integer
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear', 
    'seed': 42
}

best_result = fmin(
    fn=objective, 
    space = search_space,
    algo = tpe.suggest,
    max_evals = 50, 
    trials = Trials()
)

  0%|                                                                                                                         | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [19:26:55] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.23794                                                                                                                                          
[1]	validation-rmse:7.30145                                                                                                                                          
[2]	validation-rmse:6.26490                                                                                                                                          
[3]	validation-rmse:6.05189                                                                                                                                          
[4]	validation-rmse:5.63773                                                                                                                                          
[5]	validation-rmse:5.58216                                                                                                                                          
[6]	

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [19:33:01] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.46573                                                                                                                                          
[1]	validation-rmse:7.09065                                                                                                                                          
[2]	validation-rmse:6.66198                                                                                                                                          
[3]	validation-rmse:6.41552                                                                                                                                          
[4]	validation-rmse:6.27431                                                                                                                                          
[5]	validation-rmse:6.18424                                                                                                                                          
[6]	

...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [20:08:36] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.49836                                                                                                                                          
[1]	validation-rmse:9.04699                                                                                                                                          
[2]	validation-rmse:8.69855                                                                                                                                          
[3]	validation-rmse:8.34294                                                                                                                                          
[4]	validation-rmse:8.10415                                                                                                                                          
[5]	validation-rmse:7.91106                                                                                                                                          
[6]	

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/opt/conda/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [20:11:40] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.57156                                                                                                                                          
[1]	validation-rmse:9.14204                                                                                                                                          
[2]	validation-rmse:8.77543                                                                                                                                          
[3]	validation-rmse:8.46658                                                                                                                                          
[4]	validation-rmse:8.06167                                                                                                                                          
[5]	validation-rmse:7.82739                                                                                                                                          
[6]	

KeyboardInterrupt: 